# Spørring som både gir tekst(navn) og kode(ID) fra JSON-stat
### eksempel HS-koder utenrikshandel

### Importerer biblioteker
Bruker [pyjstat](https://pypi.org/project/pyjstat/) og pandas. Pandas lastes som del av pyjstat

In [1]:
from pyjstat import pyjstat
import requests

### URL med tabellens metadata, som vi skal poste spørringen mot

In [2]:
tabid = "08799" # kam endres til årstabellen 08801, uten andre endringer
lang = "no" # language code can also be "en"
POST_URL = "https://data.ssb.no/api/v0/" + lang + "/table/" + tabid  # 'https://data.ssb.no/api/v0/no/table/08799'

Spørring, kan tas fra konsoll: Import / eksport alle varekoder til US siste måned, ca. 65000 celler. Max grense for en spørring i PxWebApi er 800.000 celler, inkl tomme celler

In [3]:
json_q = {
  "query": [
    {
      "code": "Varekoder",
      "selection": {
        "filter": "all",
        "values": [
          "*"
        ]
      }
    },
    {
      "code": "ImpEks",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Land",
      "selection": {
        "filter": "item",
        "values": [
          "US"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "Mengde1",
          "Verdi",
          "Mengde2"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "top",
        "values": [
          "1"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}


### Poster spørringen

In [4]:
res = requests.post(POST_URL, json=json_q)

### Leser JSON-stat resultatet med biblioteket pyjstat
Lagrer det som datasett ds.

In [5]:
ds = pyjstat.Dataset.read(res.text)

In [6]:
type(ds)

pyjstat.pyjstat.Dataset

Se datasettet ds

In [7]:
# ds

### Henter noen overordnete metadata fra JSON-stat datasettet. 
#### Nytt: Flere metadata i 2024

In [8]:
tittel = ds['label']
print(tittel)

08799: Utenrikshandel med varer, etter varenummer, import/eksport, land, statistikkvariabel og måned


Sist Oppdatert som GMT

In [9]:
last_update = ds['updated']
print(last_update)

2024-12-16T07:00:00Z


Henter kilde

In [10]:
source = ds['source']
print(source)

Statistisk sentralbyrå


Roles gir noen snarveier til IDene på faste dimensjoner i datasettet

In [11]:
ds_roles = ds['role']
print(ds_roles)

OrderedDict({'time': ['Tid'], 'geo': ['Land'], 'metric': ['ContentsCode']})


Nye metadata i JSON-stat2 er tableid (tabellnummer), kort tabelltittel og noter

In [12]:
table_id = ds['extension']['px']['tableid']
print(table_id)

08799


In [13]:
kort_tit = ds['extension']['px']['contents']
print(kort_tit)

08799: Utenrikshandel med varer,


### Lager dataframes
Vi må lage to dataframes, en med tekst og en med ID. Pyjstat tillater bare enten/eller

In [14]:
hstrade = ds.write('dataframe')
hstrade_id = ds.write('dataframe', naming='id')

In [15]:
hstrade.head()

,varenummer,import/eksport,land,statistikkvariabel,måned,value
0,"Pastiller, sukkertøy og drops, uten innh av ka...",Import,USA,Mengde 1 (M1),2024M11,0
1,"Pastiller, sukkertøy og drops, uten innh av ka...",Import,USA,Verdi (kr),2024M11,0
2,"Pastiller, sukkertøy og drops, uten innh av ka...",Import,USA,Mengde 2 (M2),2024M11,0
3,"Pastiller, sukkertøy og drops, uten innh av ka...",Eksport,USA,Mengde 1 (M1),2024M11,0
4,"Pastiller, sukkertøy og drops, uten innh av ka...",Eksport,USA,Verdi (kr),2024M11,0


In [16]:
hstrade_id.head()

,Varekoder,ImpEks,Land,ContentsCode,Tid,value
0,17049092_1995,1,US,Mengde1,2024M11,0
1,17049092_1995,1,US,Verdi,2024M11,0
2,17049092_1995,1,US,Mengde2,2024M11,0
3,17049092_1995,2,US,Mengde1,2024M11,0
4,17049092_1995,2,US,Verdi,2024M11,0


### Lager en ny kolonne med både HS varekode og tekst slått sammen

In [17]:
hstrade['hstrade_combi'] = hstrade_id['Varekoder'] + ' ' + hstrade['varenummer']

In [18]:
hstrade.columns

Index(['varenummer', 'import/eksport', 'land', 'statistikkvariabel', 'måned',
       'value', 'hstrade_combi'],
      dtype='object')

### Lager en ny dataframe som bare har med kolonnene vi ønsker, i ny rekkefølge. 
Merk doble [[ ]] for å lage dataframe

In [19]:
hstrade_new = hstrade[['hstrade_combi', 'import/eksport', 'land', 'statistikkvariabel', 'måned',
       'value']]

In [20]:
hstrade_new.tail(10)

,hstrade_combi,import/eksport,land,statistikkvariabel,måned,value
79994,99319900_2007 Utenlandsk proviant/forbruksart ...,Import,USA,Mengde 2 (M2),2024M11,0
79995,99319900_2007 Utenlandsk proviant/forbruksart ...,Eksport,USA,Mengde 1 (M1),2024M11,0
79996,99319900_2007 Utenlandsk proviant/forbruksart ...,Eksport,USA,Verdi (kr),2024M11,0
79997,99319900_2007 Utenlandsk proviant/forbruksart ...,Eksport,USA,Mengde 2 (M2),2024M11,0
79998,99999999_1988 Sum alle varenumre med restriksj...,Import,USA,Mengde 1 (M1),2024M11,152028
79999,99999999_1988 Sum alle varenumre med restriksj...,Import,USA,Verdi (kr),2024M11,2000616
80000,99999999_1988 Sum alle varenumre med restriksj...,Import,USA,Mengde 2 (M2),2024M11,0
80001,99999999_1988 Sum alle varenumre med restriksj...,Eksport,USA,Mengde 1 (M1),2024M11,65335483
80002,99999999_1988 Sum alle varenumre med restriksj...,Eksport,USA,Verdi (kr),2024M11,459597701
80003,99999999_1988 Sum alle varenumre med restriksj...,Eksport,USA,Mengde 2 (M2),2024M11,0


## Fotnoter

Fotnoter er nytt i JSON-stat2. Fra SSB statstikkbank er det klare svakheter i formatering. Det er ikke entydig skilletegn notene og forstyrrende koder for visuell formatering. Det er uklart om  beste praksis vil være "nøstet liste" eller ikke.  For koden her har jeg fått bistand av "Claude".

In [21]:
if 'note' in ds:
    note = ds['note']
    print(note)

["For informasjon om gyldighet for og årlige endringer i varenummere, se  <a target='_top' href='http://www.ssb.no/emner/09/05/nos_varefortegn/main.html'>Statistisk varefortegnelse for utenrikshandelen</a>.\n<br>\n\nFra og med 2006 fikk følgende land endret tilhørighet til verdensdel - Kypros fra Asia til Europa, og Armenia, Aserbajdsjan, Georgia, Kirgisistan, Kasakhstan, Tadsjikistan, Turkmenistan og Usbekistan fra Europa til Asia. I utvalgene av enkeltland i nedtrekksmenyen for Europeiske land/Asiatiske land er det den nåværende inndelingen som vises for hele tidsperioden (Kypros hører med til Europa og de andre landene til Asia ).\n<br /><br />\nSSB publiserer ikke tall for handelsområdet tidligere Comecon etter 2011.Grupperingen og besto (i handelsstatistikken) kun av europeiske Comecon-land: Armenia, Aserbajdsjan, Bulgaria, Estland, Georgia, Hviterussland, Kasakhstan, Kirgisistan, Latvia, Litauen, Moldova, Polen, Romania, Russland, Slovakia, Tsjekkia, Tadsjikistan, Turkmenistan, U

In [22]:
# re er for regulære uttrykk - brukes for søk/erstatt
import re

def clean_text(text_list):
    """
    Fjerner \n, <br> og <br /> fra tekster
    """
    cleaned_list = []
    for text in text_list:
        # Fjern \n, <br> og <br />
        cleaned_text = re.sub(r'\\n|<br\s*/?>', ' ', text)
        
        # Fjern ekstra mellomrom
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
        
        cleaned_list.append(cleaned_text)
    
    return cleaned_list

Spitter flere fotnoter i flere lister.

In [23]:
def split_note(text_list):
    """
    Splitter tekster og returnerer separate lister
    """
    # Leter etter: " eller ' - komma mellomrom - " eller ' 
    split_pattern = r'(?:"|\')\s*(?:"|\')'
    
    # Lag en liste for alle notene
    list_of_lists = []

   # Gå gjennom hver tekst i input-listen
    for text in text_list:
        # Liste for å lagre hver fotnote
        parts_of_text = []
        
        # Del opp teksten ved anførselstegn-mønsteret
        raw_parts = re.split(split_pattern, text)
                      
        # Gå gjennom hver del
        for part in raw_parts:
            # Fjern mellomrom i start og slutt
            cleaned_part = part.strip()
            
            # Hvis delen ikke er tom etter rensing
            if cleaned_part:
                # Legg til i listen for denne teksten
                parts_of_text.append(cleaned_part)
        
        # Legg til denne tekstens deler i hovedlisten
        list_of_lists.append(parts_of_text)
    

    return list_of_lists

In [24]:
renset_note = clean_text(note)
print(renset_note)

["For informasjon om gyldighet for og årlige endringer i varenummere, se <a target='_top' href='http://www.ssb.no/emner/09/05/nos_varefortegn/main.html'>Statistisk varefortegnelse for utenrikshandelen</a>. Fra og med 2006 fikk følgende land endret tilhørighet til verdensdel - Kypros fra Asia til Europa, og Armenia, Aserbajdsjan, Georgia, Kirgisistan, Kasakhstan, Tadsjikistan, Turkmenistan og Usbekistan fra Europa til Asia. I utvalgene av enkeltland i nedtrekksmenyen for Europeiske land/Asiatiske land er det den nåværende inndelingen som vises for hele tidsperioden (Kypros hører med til Europa og de andre landene til Asia ). SSB publiserer ikke tall for handelsområdet tidligere Comecon etter 2011.Grupperingen og besto (i handelsstatistikken) kun av europeiske Comecon-land: Armenia, Aserbajdsjan, Bulgaria, Estland, Georgia, Hviterussland, Kasakhstan, Kirgisistan, Latvia, Litauen, Moldova, Polen, Romania, Russland, Slovakia, Tsjekkia, Tadsjikistan, Turkmenistan, Ukraina, Ungarn, Usbekista

In [25]:
noter = split_note(renset_note)
print(noter)

[["For informasjon om gyldighet for og årlige endringer i varenummere, se <a target='_top' href='http://www.ssb.no/emner/09/05/nos_varefortegn/main.html'>Statistisk varefortegnelse for utenrikshandelen</a>. Fra og med 2006 fikk følgende land endret tilhørighet til verdensdel - Kypros fra Asia til Europa, og Armenia, Aserbajdsjan, Georgia, Kirgisistan, Kasakhstan, Tadsjikistan, Turkmenistan og Usbekistan fra Europa til Asia. I utvalgene av enkeltland i nedtrekksmenyen for Europeiske land/Asiatiske land er det den nåværende inndelingen som vises for hele tidsperioden (Kypros hører med til Europa og de andre landene til Asia ). SSB publiserer ikke tall for handelsområdet tidligere Comecon etter 2011.Grupperingen og besto (i handelsstatistikken) kun av europeiske Comecon-land: Armenia, Aserbajdsjan, Bulgaria, Estland, Georgia, Hviterussland, Kasakhstan, Kirgisistan, Latvia, Litauen, Moldova, Polen, Romania, Russland, Slovakia, Tsjekkia, Tadsjikistan, Turkmenistan, Ukraina, Ungarn, Usbekist

In [26]:
print("Resultat: Teksten ble delt i disse delene:")
for i, sublist in enumerate(noter, 1):
    print(f"\nFotnote {i}:")
    for j, part in enumerate(sublist, 1):
        print(f"  {part}")

Resultat: Teksten ble delt i disse delene:

Fotnote 1:
  For informasjon om gyldighet for og årlige endringer i varenummere, se <a target='_top' href='http://www.ssb.no/emner/09/05/nos_varefortegn/main.html'>Statistisk varefortegnelse for utenrikshandelen</a>. Fra og med 2006 fikk følgende land endret tilhørighet til verdensdel - Kypros fra Asia til Europa, og Armenia, Aserbajdsjan, Georgia, Kirgisistan, Kasakhstan, Tadsjikistan, Turkmenistan og Usbekistan fra Europa til Asia. I utvalgene av enkeltland i nedtrekksmenyen for Europeiske land/Asiatiske land er det den nåværende inndelingen som vises for hele tidsperioden (Kypros hører med til Europa og de andre landene til Asia ). SSB publiserer ikke tall for handelsområdet tidligere Comecon etter 2011.Grupperingen og besto (i handelsstatistikken) kun av europeiske Comecon-land: Armenia, Aserbajdsjan, Bulgaria, Estland, Georgia, Hviterussland, Kasakhstan, Kirgisistan, Latvia, Litauen, Moldova, Polen, Romania, Russland, Slovakia, Tsjekkia,